In [1]:

import math
import torch 
import torch.nn as nn
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from utilities import *
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
np.random.seed(1)

train_on_gpu = torch.cuda.is_available()
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
cuda0 = torch.device('cuda:0')
dtype = torch.cuda.FloatTensor
from torch.autograd import Variable
torch.cuda.get_device_name(0)


'Quadro M1000M'

In [2]:
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

ipython = get_ipython()
if '__IPYTHON__' in globals():
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')

In [3]:
class CNNModule():


    def Conv2D(self,features=None,n_filters=None,kernal_size=None,stride=None,padding=None,layer_name=None):
        
        self.features = features
        self.n_filters = n_filters
        self.kernal_size = kernal_size
        self.stride = stride
        self.padding = padding
        self.layer_name = layer_name
        # Retrieve dimensions from A_prev's shape  
        (self.mini_batch_size, self.img_height_prev, self.img_Width_prev, self.n_channels_prev) = self.features.shape
        self.weight,self.bias = self.weight_bias_initializer(self.layer_name)
        # Retrieve dimensions from W's shape 
        (self.filter_size, self.filter_size, self.n_channels_prev, self.n_channels_new) = self.weight.shape
    

    
        # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
        self.img_height_new = int(((self.img_height_prev - self.filter_size + 2*self.padding)/self.stride) + 1)
        self.img_width_new = int(((self.img_Width_prev - self.filter_size + 2*self.padding)/self.stride) + 1)
    
        # Initialize the output volume Z with zeros.
        self.Z = np.zeros((self.mini_batch_size,self.img_height_new,self.img_width_new,self.n_channels_new))
    
        # Create A_prev_pad by padding A_prev
        self.feature_padding = np.pad(self.features,((0,0),(self.padding,self.padding),(self.padding,self.padding),(0,0)),'constant')
    

        # Select ith training example's padded activation
        for h in range(self.img_height_new):                           # loop over vertical axis of the output volume
            for w in range(self.img_width_new):                       # loop over horizontal axis of the output volume
                vert_start = h*self.stride 
                vert_end = self.filter_size+vert_start
                horiz_start = w*self.stride 
                horiz_end = self.filter_size + horiz_start
                    
                # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell).
                a_slice_prev =self.feature_padding[:,vert_start :vert_end,horiz_start:horiz_end,:]
                a_slice_prev = a_slice_prev.reshape(a_slice_prev.shape[0], -1).T
                WW=np.concatenate(np.concatenate(self.weight[:,:,:,:])).T
                SS=np.dot(WW,a_slice_prev).reshape(self.n_channels_new,self.mini_batch_size)
                self.Z[:, h, w, :]=(SS+self.bias[:,:,:,:].reshape(self.n_channels_new,1)).T



        # Making sure your output shape is correct
        assert(self.Z.shape == (self.mini_batch_size,self.img_height_new,self.img_width_new,self.n_channels_new))
    
        # Save information in "cache" for the backprop
        cache = (self.features,self.weight, self.bias,self.stride,self.padding)
    
        return self.Z, cache
        


    def MaxPool(self,conv_layer_output,pool_layer_stride,pool_layer_filter_size,mode="max"):
        
        self.conv_layer_output = conv_layer_output
        self.pool_layer_filter_size = pool_layer_filter_size
        self.pool_layer_stride = pool_layer_stride
        
        self.mode = mode
    
        # Retrieve dimensions from the input shape
        (self.mini_batch_size, self.img_height_prev, self.img_Width_prev, self.n_channels_prev) = self.conv_layer_output.shape

        # Retrieve hyperparameters from "hparameters"

        # Define the dimensions of the output
        self.img_height_new = int(1 + (self.img_height_prev - self.pool_layer_filter_size) / self.pool_layer_stride)
        self.img_width_new = int(1 + (self.img_Width_prev - self.pool_layer_filter_size) / self.pool_layer_stride)
        self.n_channels_new = self.n_channels_prev
    
        # Initialize output matrix A
        max_pooling_output = np.zeros((self.mini_batch_size, self.img_height_new, self.img_width_new,self.n_channels_new)) 
    
        for h in range(self.img_height_new):                     # loop on the vertical axis of the output volume
            for w in range(self.img_width_new):                  # loop on the horizontal axis of the output volume
                for c in range (self.n_channels_new):            # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = h*self.pool_layer_stride
                    vert_end = self.pool_layer_filter_size+vert_start
                    horiz_start = w*self.pool_layer_stride
                    horiz_end = self.pool_layer_filter_size+ horiz_start
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c.
                    a_prev_slice = self.conv_layer_output[:,vert_start :vert_end,horiz_start:horiz_end,c]
                    a_prev_slice = a_prev_slice.reshape(a_prev_slice.shape[0], -1).T
                    
                    
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    if self.mode == "max":
                        max_pooling_output[:, h, w, c] = np.max(a_prev_slice,axis=0)
                    elif self.mode == "average":
                        max_pooling_output[:, h, w, c] = np.mean(a_prev_slice,axis=0)
    
    
        # Store the input and hparameters in "cache" for pool_backward()
        cache = (self.conv_layer_output, self.pool_layer_stride,self.pool_layer_filter_size)
    
        # Making sure your output shape is correct
        assert(max_pooling_output.shape == (self.mini_batch_size, self.img_height_new, self.img_width_new,self.n_channels_new))
    
        return max_pooling_output, cache
    
    def Dropout(self,activation_output,keep_prop):
        drop_array = np.random.rand(*activation_output.shape) < keep_prop
        dropout_output = np.multiply(activation_output,drop_array)
        dropout_output /=keep_prop
        return dropout_output
    
    def Flatten(self):
        pass
    
    def Activation(self,inputs,activation_type="relu"):
        if activation_type=="softmax":
            input_shift = inputs - np.max(inputs)
            activation_output= np.exp(input_shift) / np.sum(np.exp(input_shift), axis=0,keepdims=True)
            return activation_output,inputs
        
        else:
            # relu activation function
            activation_output = np.maximum(inputs,0)
        
        return activation_output,inputs
        

    
    def weight_bias_initializer(self,layer_name):
        parameters={}
        np.random.seed(1)
        ###### initialize the weights :
    
        parameters["W"+self.layer_name] = np.random.randn(self.kernal_size,self.kernal_size,
                                                          self.n_channels_prev,self.n_filters)*np.sqrt(2/(self.n_channels_prev*(self.kernal_size)**2))
        
        parameters["b" + self.layer_name] = np.zeros((1,1,1,self.n_filters))
        return parameters["W"+self.layer_name],parameters["b" + self.layer_name]
    

## Prepare the dataset

In [4]:
def load_dataset():
    # train dataset
    train_dataset = h5py.File(r'train_signs.h5', "r")
    train_set_x_orig = torch.tensor(train_dataset["train_set_x"][:]).to(cuda0) # train set features
    train_set_y_orig = torch.LongTensor(train_dataset["train_set_y"][:]).to(cuda0) # train set labels
    
    # test dataset
    test_dataset = h5py.File(r'test_signs.h5', "r")
    test_set_x_orig = torch.tensor(test_dataset["test_set_x"][:]).to(cuda0)# test set features
    test_set_y_orig = torch.LongTensor(test_dataset["test_set_y"][:]).to(cuda0) # test set labels
    
    # classes in dataset
    classes = torch.LongTensor(test_dataset["list_classes"][:]) # the list of classes
    
    # reshape the labels in test and train dataset
    train_set_y_orig = train_set_y_orig.view(train_set_y_orig.shape[0],1)
    test_set_y_orig = test_set_y_orig.view(test_set_y_orig.shape[0],1)
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [5]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_dataset()

### Dataset in more details

##### train_set_x_orig shape = (1080, 64, 64, 3)
##### train_set_y_orig shape = (1080, 1)
##### test_set_x_orig shape  = (120, 64, 64, 3)
##### test_set_y_orig shape  = (120, 1)
##### Classes =([0, 1, 2, 3, 4, 5], dtype=int64)

## Convert the lables into one-hot-encoding

In [6]:
def convert_to_one_hot(Y, C):
    Y = torch.t(torch.eye(C)[Y.view(-1)])
    return Y.cuda()

In [7]:
train_yoh = convert_to_one_hot(train_y,len(classes))# the new shape is (1080,6) 
test_yoh  = convert_to_one_hot(test_y,len(classes))  # the new shape is (120,6)
train_yoh.type()
train_yoh.get_device()

0

## Normalizing the data 

In [8]:
train_x_orig=train_x_orig/255   # shape (1080, 64, 64, 3)
test_x_orig=test_x_orig/255    # shape (120, 64, 64, 3)

## Mini-Batch

In [9]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    torch.manual_seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(torch.randperm(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[:,permutation]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

## Initializing the Filters for Convolutional Layers

In [10]:
def initialize_parameters_he():
    
    parameters={}
    torch.manual_seed(1)
    ###### initialize the weights :
    
    parameters["W1"] = torch.randn(5,5,3,32).cuda().type(dtype)*np.sqrt(2/((3)*25))
    parameters["W2"] = torch.randn(3,3,32,64).cuda().type(dtype)*np.sqrt(2/((32)*9)) 
    parameters["W3"] = torch.randn(3,3,64,64).cuda().type(dtype)*np.sqrt(2/((64)*9))
    parameters["W4"] = torch.randn(3,3,64,64).cuda().type(dtype)*np.sqrt(2/((64)*9))
    
    # fully connected layer with 256 neurons
    parameters["W5"] = torch.randn(256,1024).cuda().type(dtype)*np.sqrt(2/((1024)))
    # output layer with 6 neurons (the number of classes)
    parameters["W6"] = torch.randn(6,256).cuda().type(dtype)*np.sqrt(2/((256)))
    
    ###### initialize the bais
    parameters["b1"] = torch.zeros((1,1,1,32)).cuda().type(dtype)
    parameters["b2"] = torch.zeros((1,1,1,64)).cuda().type(dtype)
    parameters["b3"] = torch.zeros((1,1,1,64)).cuda().type(dtype)
    parameters["b4"] = torch.zeros((1,1,1,64)).cuda().type(dtype)
    parameters["b5"] = torch.zeros((256,1)).cuda().type(dtype)
    parameters["b6"] = torch.zeros((6,1)).cuda().type(dtype)
 
    return parameters

## Initializing the  Adam Parameters ( We need that to update the weight and bias with Adam optimizer)

In [11]:
def initialize_adam(parameters) :

    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
        if(l>3):
            v["dW" + str(l+1)] = torch.zeros(( parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1])).cuda().type(dtype)
            v["db" + str(l+1)] = torch.zeros(( parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1])).cuda().type(dtype)
            s["dW" + str(l+1)] = torch.zeros(( parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1])).cuda().type(dtype)
            s["db" + str(l+1)] = torch.zeros(( parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1])).cuda().type(dtype)

        else:   
            v["dW" + str(l+1)] = torch.zeros(( parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1],parameters["W" + str(l+1)].shape[2],parameters["W" + str(l+1)].shape[3])).cuda().type(dtype)
            v["db" + str(l+1)] = torch.zeros(( parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1],parameters["b" + str(l+1)].shape[2],parameters["b" + str(l+1)].shape[3])).cuda().type(dtype)
            s["dW" + str(l+1)] = torch.zeros(( parameters["W" + str(l+1)].shape[0],parameters["W" + str(l+1)].shape[1],parameters["W" + str(l+1)].shape[2],parameters["W" + str(l+1)].shape[3])).cuda().type(dtype)
            s["db" + str(l+1)] = torch.zeros(( parameters["b" + str(l+1)].shape[0],parameters["b" + str(l+1)].shape[1],parameters["b" + str(l+1)].shape[2],parameters["b" + str(l+1)].shape[3])).cuda().type(dtype)

    return v, s

## Helper functions to apply convolution in an Image

In [14]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    X = X.transpose(1,3).transpose(2,3)
    m = nn.ZeroPad2d((pad, pad, pad, pad))
    X = m(X)
    X_pad = X.transpose(1,3).transpose(1,2)
    #X_pad = torch.from_numpy(np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values = (0,0)))
    

    return X_pad

In [15]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.

    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)

    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    # Element-wise product between a_slice and W. Do not add the bias yet.
    s = torch.mul(a_slice_prev,W)
    # Sum over all entries of the volume s.
    Z = torch.sum(s)
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z = Z+b

    return Z

In [16]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    # Retrieve dimensions from A_prev's shape  
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape 
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters" 
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Compute the dimensions of the CONV output volume using the formula given above. Hint: use int() to floor. (≈2 lines)
    n_H = int(((n_H_prev - f + 2*pad)/stride) + 1)
    n_W = int(((n_W_prev - f + 2*pad)/stride) + 1)
    
    # Initialize the output volume Z with zeros.
    Z = torch.zeros((m,n_H,n_W,n_C)).cuda().type(dtype)
    
    # Create A_prev_pad by padding A_prev
    
    A_prev_pad = zero_pad(A_prev,pad)
    

        # Select ith training example's padded activation
    for h in range(n_H):                           # loop over vertical axis of the output volume
        for w in range(n_W):   
     
            # loop over horizontal axis of the output volume
            vert_start = h*stride #stride=2
            vert_end = f+vert_start
            horiz_start = w*stride # stride=2
            horiz_end = f + horiz_start
            # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell).
            a_slice_prev =A_prev_pad[:,vert_start :vert_end,horiz_start:horiz_end,:]
            a_slice_prev = a_slice_prev.contiguous()
            a_slice_prev = a_slice_prev.view(-1,a_slice_prev.shape[0])
            WW= W.view(W.shape[-1],-1)
            SS=torch.mm(WW,a_slice_prev.type(dtype)).view(n_C,m)
            Z[:, h, w, :]=torch.t(SS+b.view(n_C,1))



    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

## Max Pooling Layer Forward

In [17]:
def pool_forward(A_prev, hparameters, mode = "max"):

    
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Initialize output matrix A
    A = torch.zeros((m, n_H, n_W, n_C)).cuda().type(dtype)
    
    for h in range(n_H):                     # loop on the vertical axis of the output volume
        for w in range(n_W):                 # loop on the horizontal axis of the output volume
            for c in range (n_C):            # loop over the channels of the output volume
                    
                # Find the corners of the current "slice" (≈4 lines)
                vert_start = h*stride
                vert_end = f+vert_start
                horiz_start = w*stride
                horiz_end = f + horiz_start
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c.
                a_prev_slice = A_prev[:,vert_start :vert_end,horiz_start:horiz_end,c]
                a_prev_slice = a_prev_slice.contiguous()
                a_prev_slice = torch.t(a_prev_slice.view(a_prev_slice.shape[0], -1))
                #print(torch.max(a_prev_slice,dim=0),np.max(a_prev_slice.cpu().numpy(),axis=0))
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                if mode == "max":
                    A[:, h, w, c] = torch.max(a_prev_slice,dim=0)[0]
                elif mode == "average":
                    A[:, h, w, c] = torch.mean(a_prev_slice,dim=0)
    
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

## Convolutional Layer Backward

In [18]:
def conv_backward(dZ, cache):


    # Retrieve information from "cache"
    (A_prev, W, b, hparameters) = cache
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters"
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = torch.zeros((m, n_H_prev, n_W_prev, n_C_prev)).cuda().type(dtype)                        
    dW = torch.zeros((f, f, n_C_prev, n_C)).cuda().type(dtype)
    db = torch.zeros((1,1,1,n_C)).cuda().type(dtype)

    # Pad A_prev and dA_prev
    A_prev_pad =zero_pad(A_prev,pad)
    dA_prev_pad = zero_pad(A_prev,pad)   
    
    # loop over the training examples
        

        
    for h in range(n_H):                   # loop over vertical axis of the output volume
        for w in range(n_W):               # loop over horizontal axis of the output volume

            vert_start = h*stride
            vert_end = f+vert_start
            horiz_start = w*stride
            horiz_end = f+ horiz_start
                    
            # Use the corners to define the slice from a_prev_pad
            a_slice =A_prev_pad[:,vert_start:vert_end,horiz_start:horiz_end,:]
  

            # Update gradients for the window and the filter's parameters using the code formulas given above
            dA_prev_pad[:,vert_start:vert_end, horiz_start:horiz_end, :] += torch.sum(W*torch.sum(dZ[:, h, w, :],dim=0).view(1,1,1,n_C),dim=-1)/m
            dW += torch.sum(a_slice.type(dtype).view(a_slice.shape[0],a_slice.shape[1],a_slice.shape[2],a_slice.shape[3],1)*dZ[:, h, w, :].view(m,1,1,1,n_C),dim=0)
            #print(a_slice*dZ[:, h, w, c].reshape(m,1,1,1))


                        
            db += torch.sum(torch.sum(dZ[:, h, w, :],dim=0).view(1,1,1,n_C),dim=-1)/n_C
            
              
                
    # Set the ith training example's dA_prev to the unpaded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
    dA_prev[:, :, :, :] = dA_prev_pad[:,pad:-pad,pad:-pad,:]

    # Making sure your output shape is correct
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    

    
    
    return dA_prev, dW, db

## Max Pooling Layer Backward

In [19]:
def create_mask_from_window(x,shape):
    """
    Creates a mask from an input matrix x, to identify the max entry of x.
    
    Arguments:
    x -- Array of shape (f, f)
    
    Returns:
    mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    """
    x=torch.t(x.contiguous().view(x.shape[0],-1))

    mask = (x==torch.max(x,dim=0)[0]).type(dtype)
    mask = mask.view(shape)

    
    return mask

In [20]:
def pool_backward(dA, cache, mode = "max"):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    

    # Retrieve information from cache 
    (A_prev, hparameters) = cache
    
    # Retrieve hyperparameters from "hparameters" 
    stride = hparameters["stride"]
    f = hparameters["f"]
    
    # Retrieve dimensions from A_prev's shape and dA's shape 
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    
    # Initialize dA_prev with zeros 
    dA_prev = torch.zeros(A_prev.shape).cuda().type(dtype)
    #a_slice_shape=[]
        # select training example from A_prev 
        
    for h in range(n_H):                   # loop on the vertical axis
        for w in range(n_W):               # loop on the horizontal axis
            for c in range(n_C):           # loop over the channels (depth)
                    
                # Find the corners of the current "slice" 
                vert_start = h*stride
                vert_end = f+ vert_start
                horiz_start = w*stride
                horiz_end = f+horiz_start
                    
                # Compute the backward propagation in both modes.
                if mode == "max":
                        
                    # Use the corners and "c" to define the current slice from a_prev 
                    a_prev_slice = A_prev[:,vert_start:vert_end,horiz_start:horiz_end,c]
                    a_slice_shape=a_prev_slice.shape

                    # Create the mask from a_prev_slice (≈1 line)
                    mask = create_mask_from_window(a_prev_slice,a_slice_shape)

                    dA_prev[:, vert_start: vert_end, horiz_start: horiz_end, c] += torch.mul(mask,dA[:,h,w,c].view(m,1,1))

                        
  
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == A_prev.shape)
    return dA_prev

## Adam Optimizer 

In [21]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):

    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    print(type(v["dW1"]),type(s["dW1"]),type(parameters["W1"]))
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
      
        v["dW" + str(l+1)] = beta1*v["dW" + str(l+1)]+ (1-beta1)*grads["dW" + str(l+1)]
        v["db" + str(l+1)] = beta1*v["db" + str(l+1)]+ (1-beta1)*grads["db" + str(l+1)]


        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
     
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)]/(1-(beta1**t))
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)]/(1-(beta1**t))
        

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        
        s["dW" + str(l+1)] = beta2*s["dW" + str(l+1)]+ (1-beta2)*(grads["dW" + str(l+1)]**2)
        s["db" + str(l+1)] = beta2*s["db" + str(l+1)]+ (1-beta2)*(grads["db" + str(l+1)]**2)
       

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".

        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)]/(1-(beta2**t))
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)]/(1-(beta2**t))
    

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*(v_corrected["dW" + str(l+1)]/(torch.sqrt(s_corrected["dW" + str(l+1)]) + epsilon))
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*(v_corrected["db" + str(l+1)]/(torch.sqrt(s_corrected["db" + str(l+1)]) + epsilon))
   

    return parameters, v, s

## Backward Propagation

In [22]:
#parameters = initialize_parameters_he()
#mini_batches=random_mini_batches(train_x_orig, train_y, mini_batch_size=64, seed=0)
#forward_prpagation(mini_batches[0][0],mini_batches[0][1],parameters)

def Model_Backward(AL,Y,caches,P4_shape,parameters,lambd):
    m=Y.shape[0]
    grads={}
    dAL=-torch.div(Y,AL)     
    
    linear_cache5,Z6=caches[5]
    A5,W6,b6=linear_cache5
    
    dZ6=softmax_backward(Z6,Y) 
    
    grads["dW6"]=(1./m)*torch.mm(dZ6,torch.t(A5)) + (lambd/m)*parameters["W6"]
    grads["db6"] = (1./m) * torch.sum(dZ6, dim = 1, keepdim = True)
    #print("grads db6 :" , grads["db6"])
    #print("grads dW6 :" , grads["dW6"])
    
    dA5 = torch.t(torch.mm(torch.t(W6),dZ6))
    
    
    linear_cache4,Z5=caches[4]
    P4,W5,b5=linear_cache4
    dZ5=relu_backward(dA5,torch.t(Z5))
    grads["dW5"]=(1./m)*torch.mm(torch.t(dZ5),torch.t(P4)) + (lambd/m)*parameters["W5"]
    grads["db5"] = (1./m) * torch.sum(torch.t(dZ5), dim = 1, keepdim = True)
    #print("grads db5 :" , grads["db5"])
    #print("grads dW5 :" , grads["dW5"])
    linear_cache3,Z4,pool_cache3=caches[3]
    
    dP4=torch.t(torch.mm(dZ5,W5))
    dP4=dP4.contiguous().view(P4_shape)
    dA4=pool_backward(dP4, pool_cache3, mode = "max")

    linear_cache2,Z3,pool_cache2=caches[2]
    
    dZ4=relu_backward(dA4, Z4)
    dP3,dW4,db4=conv_backward(dZ4, linear_cache3)
    grads["dW4"]=dW4 + (lambd/m)*parameters["W4"]
    grads["db4"]=db4
    
    dA3=pool_backward(dP3, pool_cache2, mode = "max")
    dZ3=relu_backward(dA3, Z3)
    dP2,dW3,db3=conv_backward(dZ3, linear_cache2)
    grads["dW3"]=dW3 + (lambd/m)*parameters["W3"]
    grads["db3"]=db3
    
    linear_cache1,Z2,pool_cache1=caches[1]
    
    dA2=pool_backward(dP2, pool_cache1, mode = "max")
    dZ2=relu_backward(dA2, Z2)
    dP1,dW2,db2=conv_backward(dZ2, linear_cache1)
    grads["dW2"]=dW2 + (lambd/m)*parameters["W2"]
    grads["db2"]=db2
    
    linear_cache0,Z1,pool_cache0=caches[0]
    
    dA1=pool_backward(dP1, pool_cache0, mode = "max")
    dZ1=relu_backward(dA1, Z1)
    dA,dW1,db1=conv_backward(dZ1, linear_cache0)
    grads["dW1"]=dW1 + (lambd/m)*parameters["W1"]
    grads["db1"]=db1
    
    assert(b6.shape == grads["db6"].shape)
    assert(b5.shape == grads["db5"].shape)
    return grads

## Forward Propagation

def forward_prpagation(X,Y,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    W6 = parameters['W6']

    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    b4 = parameters['b4']
    b5 = parameters['b5']
    b6 = parameters['b6']
    caches=[]
    
    #Convolution layer 1
    model = CNNModule()
    Z1, linear_cache = model.Conv2D(X, n_filters=32, kernal_size=5, stride=1,padding=2,layer_name="Conv_layer_1")
    A1, activation_cache = model.Activation(Z1,activation_type="relu")
    A1 = model.Dropout(A1,keep_prop=0.7)
    P1,linear_pool_cache=model.MaxPool(A1,pool_layer_stride=2,pool_layer_filter_size=2, mode = "max")

    
    
    caches.append((linear_cache,activation_cache,linear_pool_cache))
    
    
    #Convolution layer 2
    Z2,linear_cache=model.Conv2D(P1, n_filters=64, kernal_size=3, stride=1,padding=1,layer_name="Conv_layer_2")
    A2,activation_cache = model.Activation(Z2,activation_type="relu")
    A2 = model.Dropout(A2,keep_prop=0.60)
    P2,linear_pool_cache=model.MaxPool(A2, pool_layer_stride=2,pool_layer_filter_size=2, mode = "max")
    #print(P2.shape)
    
    caches.append((linear_cache,activation_cache,linear_pool_cache))
    
    
    #Convolution layer 3
    Z3,linear_cache=model.Conv2D(P2, n_filters=64, kernal_size=3, stride=1,padding=1,layer_name="Conv_layer_3")
    A3,activation_cache = model.Activation(Z3,activation_type="relu")
    A3 = model.Dropout(A3,keep_prop=0.8)
    P3,linear_pool_cache=model.MaxPool(A3, pool_layer_stride=2,pool_layer_filter_size=2, mode = "max")

    
    caches.append((linear_cache,activation_cache,linear_pool_cache))
    
    #Convolution layer 4
    Z4,linear_cache=model.Conv2D(P3, n_filters=64, kernal_size=3, stride=1,padding=1,layer_name="Conv_layer_4")
    A4,activation_cache = model.Activation(Z4,activation_type="relu")
    A4 = model.Dropout(A4,keep_prop=0.6)
    P4,linear_pool_cache=model.MaxPool(A4, pool_layer_stride=2,pool_layer_filter_size=2, mode = "max")

    
    caches.append((linear_cache,activation_cache,linear_pool_cache))
    
    #flatten
    P4_shape=P4.shape[:]

    P4 = P4.reshape(P2.shape[0], -1).T

    #Fully_Connected
    
    Z5,linear_cache= linear_forward(P4,W5,b5)
    A5,activation_cache = model.Activation(Z5,activation_type="relu")
    A5 = model.Dropout(A5,keep_prop=0.8)
    caches.append((linear_cache,activation_cache))
    
    Z6,linear_cache=linear_forward(A5,W6,b6)
    A6,activation_cache = model.Activation(Z6,activation_type="softmax")
    caches.append((linear_cache,activation_cache))
    cost,AL = cost_function(A6,Y,parameters,lambd = 0.1)
    #print(sum(A3[0,:]))
    return AL,caches,P4_shape,cost

In [23]:
def forward_prpagation(X,Y,parameters):
    W1 = parameters['W1']
    W2 = parameters['W2'] 
    W3 = parameters['W3'] 
    W4 = parameters['W4'] 
    W5 = parameters['W5'] 
    W6 = parameters['W6']

    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    b4 = parameters['b4']
    b5 = parameters['b5']
    b6 = parameters['b6']
    caches=[]
    #Convolution layer 1
    Z1, linear_cache = conv_forward(X,W1, b1, hparameters={"stride":1,"pad":2})
    A1, activation_cache = relu(Z1)
    A1 = Dropout(A1,keep_prop=0.7)
    P1,linear_pool_cache=pool_forward(A1, hparameters={"stride":2,"f":2}, mode = "max")



    caches.append((linear_cache,activation_cache,linear_pool_cache))


    #Convolution layer 2
    Z2,linear_cache=conv_forward(P1,W2, b2, hparameters={"stride":1,"pad":1})
    A2,activation_cache = relu(Z2)
    A2 = Dropout(A2,keep_prop=0.60)
    P2,linear_pool_cache=pool_forward(A2, hparameters={"stride":2,"f":2}, mode = "max")
    #print(P2.shape)

    caches.append((linear_cache,activation_cache,linear_pool_cache))


    #Convolution layer 3
    Z3,linear_cache=conv_forward(P2,W3, b3, hparameters={"stride":1,"pad":1})
    A3,activation_cache = relu(Z3)
    A3 = Dropout(A3,keep_prop=0.8)
    P3,linear_pool_cache=pool_forward(A3, hparameters={"stride":2,"f":2}, mode = "max")


    caches.append((linear_cache,activation_cache,linear_pool_cache))

    #Convolution layer 4
    Z4,linear_cache=conv_forward(P3,W4, b4, hparameters={"stride":1,"pad":1})
    A4,activation_cache = relu(Z4)
    A4 = Dropout(A4,keep_prop=0.6)
    P4,linear_pool_cache=pool_forward(A4, hparameters={"stride":2,"f":2}, mode = "max")


    caches.append((linear_cache,activation_cache,linear_pool_cache))

    #flatten
    P4_shape=P4.shape[:]

    P4 = torch.t(P4.view(P2.shape[0], -1))

    #Fully_Connected

    Z5,linear_cache= linear_forward(P4,W5,b5)
    A5,activation_cache = relu(Z5)
    A5 = Dropout(A5,keep_prop=0.8)
    caches.append((linear_cache,activation_cache))

    Z6,linear_cache=linear_forward(A5,W6,b6)
    A6,activation_cache = softmax(Z6)
    caches.append((linear_cache,activation_cache))
    cost,AL = cost_function(A6,Y,parameters,lambd = 0.1)
    #print(sum(A3[0,:]))
    return AL,caches,P4_shape,cost

## Final Model

In [24]:
def Convolution_model(X, Y,mini_batch_size, learning_rate = 0.001, beta = 0.9,beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_iterations= 100, print_cost = True):#lr was 0.009

    t = 0
    c=0
    seed=10
    costs = []                     # keep track of cost
    
    # Parameters initialization.

    parameters = initialize_parameters_he() 
    v,s= initialize_adam(parameters)
    # Loop (gradient descent)
    for i in range(0, num_iterations):
        seed+=1
        mini_batches=random_mini_batches(X, Y, mini_batch_size, seed)
        for j in range(len(mini_batches)):
            (mini_batch_x,mini_batch_y)=mini_batches[j]
            (mini_batch_x,mini_batch_y) = (Variable(mini_batch_x.cuda()).type(dtype),Variable(mini_batch_y.cuda()).type(dtype))
            #mini_batch_y = convert_to_one_hot(mini_batch_y,6) 
 
        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
            AL,caches,P2_shape,cost= forward_prpagation(mini_batch_x,mini_batch_y,parameters)



        # Backward propagation.
            print("backward_beginn")
            grads = Model_Backward(AL,mini_batch_y,caches,P2_shape,parameters,lambd = 0.1)
            c+=1
            #grads = conv_backward(grads_Z, caches)
            print("backward_end")
            #difference = gradient_check_n(parameters, grads, mini_batch_x, mini_batch_y)
        # Update parameters.
            t = t + 1 # Adam counter
            parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,t, learning_rate, beta1, beta2,  epsilon)
        
        # Visualizaing the feature maps

            
 

                
        # Print the cost every 100 training example
        if print_cost and i % 1 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            #predictions_train = predict(train_x_orig, train_yoh, parameters)
            costs.append(cost)

            
            
    #plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    """
    W1 = parameters['W1']
    W2 = parameters['W2']
    fig,axes = plt.subplots(nrows = 2, ncols = 4, figsize=(20,20))
    for i in range(2):
        for j in range(4):
            axes[i,j].imshow(W1[:,:,:,(i*4)+j])
    """
    return parameters

In [25]:
parameters = Convolution_model(train_x_orig, train_yoh,mini_batch_size=4, num_iterations = 3, print_cost = True)

backward_beginn
backward_end
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
backward_beginn


KeyboardInterrupt: 